# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply?

A two-sample hypothesis test of proportions is appropriate.  

Since the number of calls and no-calls are greater than 10 for both the black and white races, CLT applies and the difference between the two proportions is approximately normal.

### 2. What are the null and alternate hypotheses?

Since we want to know whether race has a significant impact, a two-sided hypothesis seems appropriate.  

The hypothesis test is:
$H_o: p_b - p_w = 0$ vs. $H_a: p_b - p_w \ne 0$  

The test size $(\alpha)$ of 0.05 is used.

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

The permutation method is used because it is a hypothesis testing.

First, determine the number of callback for black and white sounding names:

In [81]:
#number of data for black-sounding names
w = len(data[data.race =='b'])
#number of data for white-sounding names
b = len(data[data.race =='w'])
print (f'The number of data for black-sounding names and white-sounding names are {b} and {w},respectively.')

# number of callbacks for black-sounding names
call_b = sum(data[data.race=='b'].call)
# number of callbacks for white-sounding names
call_w = sum(data[data.race=='w'].call)
print (f'The number of callbacks for black-sounding names and white-sounding names are {int(call_b)} and {int(call_w)},respectively.')

The number of data for black-sounding names and white-sounding names are 2435 and 2435,respectively.
The number of callbacks for black-sounding names and white-sounding names are 157 and 235,respectively.


Determine the proportion and difference:

In [52]:
p_b = call_b/b
p_w = call_w/w 
diff_race_obs = p_b - p_w

print(f'The proportion of blacks and whites with callbacks are {p_b:.3} and {p_w:.3}, respectively.')
print(f'The proportion difference is {diff_race_obs:.3}.')

The proportion of blacks and whites with callbacks are 0.0645 and 0.0965, respectively.
The proportion difference is -0.032.


Perform the permutation simulation:

In [53]:
np.random.seed(100)
n_reps = 10000
diff_race_perm = np.empty(n_reps)
for i in range(n_reps):
    data_perm = np.random.permutation(data.call)
    black_perm = sum(data_perm[:b])/b
    white_perm = sum(data_perm[b:])/w
    diff_race_perm[i] = black_perm - white_perm

p_value = (sum(diff_race_perm <= diff_race_obs) + sum(diff_race_perm >= -1*diff_race_obs))/len(diff_race_perm)
print(f'The p-value is: {p_value}')

The p-value is: 0.0


The p-value obtained from this simulation is 0.0. We reject the null hypothesis and conclude that there is significant difference in callback rates between blacks and white sounding names.

Next we use the frequentist approach. The standard error of $\hat{p}_b - \hat{p}_w$ is:  
$\sqrt{\frac{\hat{p}_b(1-\hat{p}_b)}{n_b} + \frac{\hat{p}_w(1-\hat{p}_w)}{n_w}}$

In [55]:
std_error = np.sqrt(p_b*(1-p_b)/b + p_w*(1-p_w)/w)
std_error

0.0077833705866767544

Margin of error:

In [73]:
print(f'At 95% confidence, the margin of error is {1.96*std_error:.3}.')

At 95% confidence, the margin of error is 0.0153.


95% Confidence Interval:

In [70]:
print(f'The 95% CI of the difference in proportion is: ({p_b-p_w - 1.96*std_error:.3},{p_b-p_w +1.96*std_error:.3}).')

The 95% CI of the difference in proportion is: (-0.0473,-0.0168).


p-value:

In [74]:
# import required package 
from scipy.stats import norm

In [80]:
p_value = 2 * norm.cdf(diff_race_obs,0,std_error)
print(f'The p-value is: {p_value:.3}.')

The p-value is: 3.86e-05


With $p-value$ < 0.05, we reject the null hypothesis. The difference in callback rate between black and white sounding names is significant.

### 4. Write a story describing the statistical significance in the context or  original problem.

In this study, we examine the difference in proportion for callback for interivews between resumes with black and white sounding names.   

Through hypothesis testing, we identified that there is a statistically signficiant difference in call back rates between the two groups of resumes.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This study only serve to identify race/name as a factor that is statistically significant in affecting callback for interviews. It does not evaluate other factors, and so we cannot conclude that name/race is "the most important factor".  

To amend the analysis, we should look at other factors that may affect callback rate, such as level of education, gender, and skills. 